# Palabras clave (1ª versión)

Este es el primer cuaderno de la presentación [Text mining is Not Magic](https://docs.google.com/presentation/d/15l5MarIFXz67HqtSoqYEGLTioNt_D5F7YPuMIB4EvIU)

## Frecuencia de aparición

In [ ]:
from collections import Counter

def get_keywords(words):
    '''Return the top10 frequent words.
    '''
    return [
        x for x,_ in 
        sorted(
            Counter(words).iteritems(),
            key=lambda (_,freq):freq,
            reverse=True
        )
    ][:10]

In [ ]:
print get_keywords(['esta', 'es', 'una', 'prueba', 'para', 'ver', 'si', 'es', 'correcta', 'la', 'función', 'para', 'calcular', 'frecuencias'])

## Carga de documentos

Dataset URL: http://www.lsi.us.es/~fermin/corpusCine.zip

Nos quedamos con los ficheros .xml, que tienen una estructura del estilo:

```xml
<review author="XXX" title="XXX" rank="X" maxRank="X" source="XXX">
	<summary>XXX</summary>
	<body>XXX</body>
</review>
```

In [ ]:
import os
import config

### Detección de la codificación

In [ ]:
import chardet

with open(config.DATASET_MUCHOCINE_RAW+'/100.xml', 'r') as fd:
    txt = fd.read()
    
print chardet.detect(txt)

### Parseado con expresiones regulares

In [ ]:
import codecs  # Know your encoding
import re

def parse_file(file_path):
    retval = {'file': os.path.basename(file_path)}
    with codecs.open(file_path,'r', encoding='ISO-8859-2') as fd:
        f = fd.read().strip()
    reg_expr = re.compile(r'\<review author="(?P<author>.*)" title="(?P<title>.*)" rank="(?P<rank>\d)".*\>\s*<summary>(?P<summary>.*)</summary>\s*<body>(?P<body>.*)</body>\s*</review>')
    regexp_result = reg_expr.search(f.strip())
    for key in ['author', 'rank', 'title', 'summary', 'body']:
        retval[key] = regexp_result.group(key)
    return retval

### Carga

In [ ]:
documents = []
for file_name in os.listdir(config.DATASET_MUCHOCINE_RAW):
    try:
        documents.append(
            parse_file(config.DATASET_MUCHOCINE_RAW+'/'+file_name)
        )
    except Exception as e:
        print file_name
        print e.message
documents.sort(key=lambda x: x['file'])

## Documento de ejemplo

In [ ]:
my_document = documents[1478]  # 'Harry Potter y la piedra filosofal'
print my_document['title']
print my_document['body']

## 1ª versión. Contar palabras

In [ ]:
def txt2words1(txt):
    words = [w for w in txt.split(' ') if w!='']
    return words

In [ ]:
print get_keywords(txt2words1(my_document['body']))

## 2ª versión. Stopwords

In [ ]:
## Taken from NLTK corpus so you don't have to download

#import nltk
#nltk.download()
#from nltk.corpus import stopwords
#print filter_symbols(
#    ' '.join(
#        sorted(stopwords.words('spanish'))
#    )
#)

STOPWORDS = set('''
a al algo algunas algunos ante antes como con contra cual cuando de del desde donde durante e el ella ellas ellos
en entre era erais eran eras eres es esa esas ese eso esos esta estaba estabais estaban estabas estad estada estadas
estado estados estamos estando estar estaremos estara estaran estaras estare estareis estaria estariais estariamos
estarian estarias estas este estemos esto estos estoy estuve estuviera estuvierais estuvieran estuvieras estuvieron
estuviese estuvieseis estuviesen estuvieses estuvimos estuviste estuvisteis estuvieramos estuviesemos estuvo esta
estabamos estais estan estas este esteis esten estes fue fuera fuerais fueran fueras fueron fuese fueseis fuesen fueses
fui fuimos fuiste fuisteis fueramos fuesemos ha habida habidas habido habidos habiendo habremos habra habran habras
habre habreis habria habriais habriamos habrian habrias habeis habia habiais habiamos habian habias han has hasta
hay haya hayamos hayan hayas hayais he hemos hube hubiera hubierais hubieran hubieras hubieron hubiese hubieseis
hubiesen hubieses hubimos hubiste hubisteis hubieramos hubiesemos hubo la las le les lo los me mi mis mucho muchos
muy mas mi mia mias mio mios nada ni no nos nosotras nosotros nuestra nuestras nuestro nuestros o os otra otras otro
otros para pero poco por porque que quien quienes que se sea seamos sean seas sentid sentida sentidas sentido sentidos
seremos sera seran seras sere sereis seria seriais seriamos serian serias seais siente sin sintiendo sobre sois somos
son soy su sus suya suyas suyo suyos si tambien tanto te tendremos tendra tendran tendras tendre tendreis tendria
tendriais tendriamos tendrian tendrias tened tenemos tenga tengamos tengan tengas tengo tengais tenida tenidas tenido
tenidos teniendo teneis tenia teniais teniamos tenian tenias ti tiene tienen tienes todo todos tu tus tuve tuviera
tuvierais tuvieran tuvieras tuvieron tuviese tuvieseis tuviesen tuvieses tuvimos tuviste tuvisteis tuvieramos tuviesemos
tuvo tuya tuyas tuyo tuyos tu un una uno unos vosostras vosostros vuestra vuestras vuestro vuestros y ya yo el eramos
'''.split())

def txt2words2(txt):
    words = [w for w in txt.split(' ') if w!='' and w not in STOPWORDS]
    return words

In [ ]:
print get_keywords(txt2words2(my_document['body']))

## 3ª versión. Quitando mayúsculas, tildes...

In [ ]:
def txt2words3(txt):
    txt = txt.lower()  # Text in lowercase
    table = dict(zip( #  Quitar tildes
        [ord(x) for x in u'áéíóúü'],
        [ord(x) for x in u'aeiouu']
    ))
    txt = txt.translate(table)    
    txt = ''.join([
        letter for letter in txt 
        if letter in set(u'abcdefghijklmnñopqrstuvwxyz0123456789 ')]
    )
    words = [w for w in txt.split(' ') if w!='' and w not in STOPWORDS]
    return words

In [ ]:
print get_keywords(txt2words3(my_document['body']))

## 4ª versión. Stemming

In [ ]:
from nltk.stem.snowball import SpanishStemmer
stemmer = SpanishStemmer()

def txt2words4(txt):
    txt = txt.lower()  # Text in lowercase
    table = dict(zip( #  Quitar tildes
        [ord(x) for x in u'áéíóúü'],
        [ord(x) for x in u'aeiouu']
    ))
    txt = txt.translate(table)    
    txt = ''.join([
        letter for letter in txt 
        if letter in set(u'abcdefghijklmnñopqrstuvwxyz0123456789 ')]
    )
    words = [
        stemmer.stem(w)
        for w in txt.split(' ')
        if w!='' and w not in STOPWORDS
    ]
    return words

In [ ]:
print get_keywords(txt2words4(my_document['body']))

## Aplicándolo a todos los documentos

In [ ]:
for d in documents:
    for key in ['body', 'summary']:
        d[key+'_tokens'] = txt2words4(d[key])

### Exploración

In [ ]:
# Film titles
titles = sorted(list({d['title'] for d in documents}))
print '\n'.join(titles)

In [ ]:
def explore(title):
    print "Título: {}".format(title)
    for kws in [get_keywords(d['body_tokens']) for d in documents if title==d['title']]:
        print ' ->  {}'.format(', '.join(kws))

In [ ]:
explore(u'Harry Potter y la piedra filosofal')

In [ ]:
explore('High School Musical')

In [ ]:
explore('Los puentes de Madison')

In [ ]:
explore('2001, Odisea del espacio')

In [ ]:
explore('Alien vs Predator 2')

In [ ]:
explore('Apocalypto')

## Exportar/Importar

In [ ]:
import json
import zlib


# Export
with open(config.DATASET_MUCHOCINE, 'w+') as fd:
    fd.write(zlib.compress(json.dumps(documents)))
    
# Import
with open(config.DATASET_MUCHOCINE, 'r') as fd:
    docs_import =json.loads(zlib.decompress(fd.read()))